In [49]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import wikipedia as wp
import yfinance as yf 
from ta.momentum import rsi 
from ta.trend import macd

In [50]:
# Try different page names for S&P 500
try:
    html = wp.page("List of S&P 500 companies").html().encode("UTF-8")
except:
    try:
        html = wp.page("S&P 500 Index").html().encode("UTF-8")
    except:
        html = wp.page("S&P 500").html().encode("UTF-8")

# Try different table indices to find the right one
tables = pd.read_html(html)
print(f"Found {len(tables)} tables")

# Usually the main S&P 500 list is in the first table (index 0)
stocks = tables[0].set_index("Symbol")
stocks = stocks.drop("GOOG", errors='ignore')  # ignore if GOOG doesn't exist
# stocks.loc["GOOGL", "Security"] = "Alphabet Inc." if "Security" in stocks.columns else stocks.loc["GOOGL", "Name"] = "Alphabet"
stocks = stocks.rename(index={"BRK.B": "BRK-B"}, errors='ignore')
stocks.head()

Found 2 tables


,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
Symbol,,,,,,,
MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [53]:
stocks[['GICS Sector']].to_csv('../input/sp500_2009_2025/domain.csv')

In [ ]:
fundamentals = [
	yf.Ticker(stock).info for stock in stocks.index
]
fundamentals = pd.DataFrame(fundamentals).set_index("symbol")
fundamentals.index = fundamentals.index.rename("Symbol")  # for consistency with the stocks dataframe

In [20]:
fundamentals.columns

Index(['address1', 'city', 'state', 'zip', 'country', 'phone', 'website',
       'industry', 'industryKey', 'industryDisp',
       ...
       'regularMarketChangePercent', 'regularMarketPrice', 'trailingPegRatio',
       'address2', 'displayName', 'fax', 'ipoExpectedDate', 'prevName',
       'nameChangeDate', 'industrySymbol'],
      dtype='object', length=185)

In [21]:
fundamentals

,address1,city,state,zip,country,phone,website,industry,industryKey,industryDisp,sector,sectorKey,sectorDisp,longBusinessSummary,fullTimeEmployees,companyOfficers,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,governanceEpochDate,compensationAsOfEpochDate,irWebsite,executiveTeam,maxAge,priceHint,previousClose,open,dayLow,dayHigh,regularMarketPreviousClose,regularMarketOpen,regularMarketDayLow,regularMarketDayHigh,dividendRate,dividendYield,exDividendDate,payoutRatio,fiveYearAvgDividendYield,...,regularMarketChange,regularMarketDayRange,fullExchangeName,averageDailyVolume3Month,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekChangePercent,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,earningsCallTimestampStart,earningsCallTimestampEnd,isEarningsDateEstimate,epsTrailingTwelveMonths,epsForward,epsCurrentYear,priceEpsCurrentYear,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,sourceInterval,exchangeDataDelayedBy,averageAnalystRating,cryptoTradeable,marketState,regularMarketChangePercent,regularMarketPrice,trailingPegRatio,address2,displayName,fax,ipoExpectedDate,prevName,nameChangeDate,industrySymbol
Symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MMM,3M Center,Saint Paul,MN,55144-1000,United States,651 733 1110,https://www.3m.com,Conglomerates,conglomerates,Conglomerates,Industrials,industrials,Industrials,3M Company provides diversified technology ser...,61500.0,"[{'maxAge': 1, 'name': 'Mr. William M. Brown',...",1.0,7.0,5.0,4.0,4.0,1.754006e+09,1.735603e+09,http://phx.corporate-ir.net/phoenix.zhtml?c=80...,[],86400.0,2,154.34,154.12,153.2050,155.3050,154.34,154.12,153.2050,155.3050,2.92,1.89,1.756080e+09,0.3972,4.12,...,0.410004,153.205 - 155.305,NYSE,3413033.0,32.769997,0.268651,121.98 - 164.15,-9.399994,-0.057265,20.334375,1.757635e+09,1.752842e+09,1.752842e+09,1.752842e+09,1.752844e+09,1.752844e+09,False,7.20,7.90,7.88836,19.617512,3.333404,0.022015,11.444046,0.079857,15,0,2.1 - Buy,False,PRE,0.265650,154.75,3.5243,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOS,11270 West Park Place,Milwaukee,WI,53224,United States,414 359 4000,https://www.aosmith.com,Specialty Industrial Machinery,specialty-industrial-machinery,Specialty Industrial Machinery,Industrials,industrials,Industrials,A. O. Smith Corporation manufactures and marke...,12700.0,"[{'maxAge': 1, 'name': 'Mr. Stephen M. Shafer'...",4.0,10.0,7.0,10.0,10.0,1.754006e+09,1.735603e+09,http://investor.shareholder.com/aosmith/,[],86400.0,2,72.82,72.64,71.8450,73.0400,72.82,72.64,71.8450,73.0400,1.36,1.89,1.753920e+09,0.3733,1.71,...,-0.970001,71.845 - 73.04,NYSE,1684848.0,13.019997,0.221316,58.83 - 92.06,-20.210000,-0.219531,-11.438429,1.755216e+09,1.753360e+09,1.761050e+09,1.761050e+09,1.753366e+09,1.753366e+09,True,3.59,4.07,3.82958,18.761848,3.067795,0.044602,3.378151,0.049336,15,0,2.6 - Hold,False,PRE,-1.332050,71.85,1.9581,Suite 170,A. O. Smith,NaN,NaN,NaN,NaN,NaN
ABT,100 Abbott Park Road,North Chicago,IL,60064-6400,United States,224 667 6100,https://www.abbott.com,Medical Devices,medical-devices,Medical Devices,Healthcare,healthcare,Healthcare,"Abbott Laboratories, together with its subsidi...",114000.0,"[{'maxAge': 1, 'name': 'Mr. Robert B. Ford', '...",9.0,7.0,9.0,2.0,7.0,1.754006e+09,1.672445e+09,http://www.abbottinvestor.com/phoenix.zhtml?c=...,[],86400.0,2,131.25,132.05,131.7200,133.8100,131.25,132.05,131.7200,133.8100,2.36,1.78,1.752538e+09,0.2861,1.68,...,1.190000,131.72 - 133.81,NYSE,6518717.0,21.580002,0.194660,110.86 - 141.23,-8.789993,-0.062239,18.144512,1.755216e+09,1.752755e+09,1.760531e+09,1.760531e+09,1.752757e+09,1.752757e+09,True,7.97,5.16,5.14258,25.753610,1.133408,0.008632,5.699455,0.044969,15,0,1.8 - Buy,False,PRE,0.906668,132.44,NaN,Abbott Park,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,1 North Waukegan Road,North Chicago,IL,60064-6400,Unite

In [22]:
fundamentals = fundamentals[["marketCap", "trailingPE", "forwardPE", "priceToBook", "trailingEps", "forwardEps", "bookValue", "payoutRatio", "beta", "fiveYearAvgDividendYield", "52WeekChange", "averageVolume", "enterpriseToRevenue", "profitMargins"]].fillna(0)
fundamentals.head()

,marketCap,trailingPE,forwardPE,priceToBook,trailingEps,forwardEps,bookValue,payoutRatio,beta,fiveYearAvgDividendYield,52WeekChange,averageVolume,enterpriseToRevenue,profitMargins
Symbol,,,,,,,,,,,,,,
MMM,8.242450e+10,21.493055,19.588608,19.214056,7.20,7.90,8.054,0.3972,1.104,4.12,0.203344,3413033.0,3.742,0.16011
AOS,1.006834e+10,20.013927,17.653560,5.454338,3.59,4.07,13.173,0.3733,1.220,1.71,-0.114384,1684848.0,5.354,0.13683
ABT,2.305065e+11,16.617315,25.666668,4.558566,7.97,5.16,29.053,0.2861,0.705,1.68,0.181445,6518717.0,5.497,0.32427
ABBV,3.694760e+11,100.071770,17.242374,-2011.057600,2.09,12.13,-0.104,3.0381,0.503,3.89,0.065081,5664393.0,7.434,0.06451
ACN,1.588587e+11,20.290375,18.127222,5.198207,12.57,14.07,49.065,0.4558,1.290,1.41,-0.228454,3973979.0,2.313,0.11607


In [23]:
fundamentals.shape

(502, 14)

In [24]:
normalized_fundamentals = (fundamentals - fundamentals.mean()) / fundamentals.std()

In [ ]:
normalized_fundamentals.to_csv("../input/sp500_2009_2025/fundamentals.csv")

In [54]:
fundamentals_df = pd.read_csv('../input/sp500_2009_2025/fundamentals.csv', index_col=0)
stocks = (fundamentals_df.index).to_list()
# Remove the Stock which doesn't have relevant data
stocks.remove('BF.B')

In [55]:
values = [
	yf.Ticker(stock).history(period="15y", actions=False) for stock in stocks
]

values

[                                 Open        High         Low       Close  \
 Date                                                                        
 2010-08-23 00:00:00-04:00   43.121715   43.808380   43.063162   43.158974   
 2010-08-24 00:00:00-04:00   42.690560   43.116403   42.301981   42.823635   
 2010-08-25 00:00:00-04:00   42.589401   43.169607   42.301957   42.983303   
 2010-08-26 00:00:00-04:00   43.254800   43.292059   42.317950   42.466995   
 2010-08-27 00:00:00-04:00   42.647954   43.164285   42.158235   43.116379   
 ...                               ...         ...         ...         ...   
 2025-08-14 00:00:00-04:00  159.600006  160.070007  156.630005  156.649994   
 2025-08-15 00:00:00-04:00  157.369995  157.369995  151.610001  152.389999   
 2025-08-18 00:00:00-04:00  152.360001  155.429993  152.190002  152.860001   
 2025-08-19 00:00:00-04:00  152.839996  155.720001  152.050003  154.339996   
 2025-08-20 00:00:00-04:00  154.119995  155.339996  153.169998  

In [56]:
sp500_df = pd.concat(values, keys=stocks)
sp500_df

Open        High         Low       Close  \
    Date                                                                        
MMM 2010-08-23 00:00:00-04:00   43.121715   43.808380   43.063162   43.158974   
    2010-08-24 00:00:00-04:00   42.690560   43.116403   42.301981   42.823635   
    2010-08-25 00:00:00-04:00   42.589401   43.169607   42.301957   42.983303   
    2010-08-26 00:00:00-04:00   43.254800   43.292059   42.317950   42.466995   
    2010-08-27 00:00:00-04:00   42.647954   43.164285   42.158235   43.116379   
...                                   ...         ...         ...         ...   
ZTS 2025-08-14 00:00:00-04:00  151.570007  153.360001  151.169998  153.059998   
    2025-08-15 00:00:00-04:00  153.720001  155.100006  152.979996  154.110001   
    2025-08-18 00:00:00-04:00  154.000000  154.940002  151.600006  152.320007   
    2025-08-19 00:00:00-04:00  152.919998  155.699997  152.509995  155.669998   
    2025-08-20 00:00:00-04:00  156.339996  160.479996  156.339996  156.759995   

                                Volume  
    Date                                
MMM 2010-08-23 00:00:00-04:00  3879106  
    2010-08-24 00:00:00-04:00  5353176  
    2010-08-25 00:00:00-04:00  4278212  
    2010-08-26 00:00:00-04:00  4278092  
    2010-08-27 00:00:00-04:00  4121296  
...                                ...  
ZTS 2025-08-14 00:00:00-04:00  2565100  
    2025-08-15 00:00:00-04:00  3705700  
    2025-08-18 00:00:00-04:00  2158500  
    2025-08-19 00:00:00-04:00  2594100  
    2025-08-20 00:00:00-04:00  3050400  

[1794723 rows x 5 columns]

In [57]:
sp500_df.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [58]:
sp500_df.to_csv('../input/sp500_2010_2025/sp500_prices_newversion.csv')